In [1]:
import os, json
import sys
import numpy as np

sys.path.append('..')

from extractEmbeddings import featureExtractor
from utils import datasetUtils
from extractEmbeddings import fvsOperations

setGPU: Setting GPU to: 0


/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3

In [2]:
featExt = featureExtractor(1000, "ENB7")
model = featExt.pretrainedModel()

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
dataUtils = datasetUtils()
fvsOps = fvsOperations()

In [4]:
def testFramesFvs(path, isSignedMagSep, isOriginal):
    testFvs, testFileNames = featExt.extractFeaturesTitle(testCasePath, model) #extract features from test case folder
    if isOriginal:
        return testFvs
    arrTestFvs = np.asarray(testFvs).squeeze() #convert list to nd.array    
    if isSignedMagSep == True:
        arrTestFvsCombined = np.apply_along_axis(fvsOps.signedmag, 0, arrTestFvs) #combined feature vectors
    else:
        arrTestFvsCombined = np.linalg.norm(arrTestFvs, axis=0)
        
    return arrTestFvsCombined

In [5]:
def parseTestCases(path):
    jsonFilePath = os.path.join(testCasePath, 'retrievaldata.json')
    if not os.path.exists(jsonFilePath):
        print("file does not exist")
    with open(jsonFilePath) as jfile:
        data = json.load(jfile)
    res = data['response']['Results']
    title = res[0].split('_')[0]
    ind = res[0].split('_')[-1]
    
    return title, ind

In [8]:
testCasesDataPath = '../../../data/movie_titles/testUncompressed/'
titlesPath = '../../../data/movie_titles/all_titles_uncompressed/'
isSignedMagSep = True
isOriginal = False

for testCase in os.listdir(testCasesDataPath):
    testCasePath = os.path.join(testCasesDataPath, testCase)
    arrTestFvs = testFramesFvs(testCasePath, isSignedMagSep, isOriginal)
    
    titleName, aInd = parseTestCases(testCasePath)
    titlePath = os.path.join(titlesPath, titleName)
    matFile = titleName+'AggregateENB7.mat'
    matFilePath = os.path.join(titlePath, matFile)
    arrFvsDB = dataUtils.loadAggMatFile(matFilePath)
    
    dist, ind = fvsOps.compareCombinedFvsFromFile(arrTestFvs, arrFvsDB)
    print("title:{}, distance:{}, predicted index:{}, actual index:{}".format(titleName, dist, ind, aInd))

title:5454, distance:0.15680283308029175, predicted index:833, actual index:832
title:28362, distance:0.12636256217956543, predicted index:1194, actual index:1194
title:180, distance:0.3014116883277893, predicted index:374, actual index:372
title:166, distance:0.22307610511779785, predicted index:1702, actual index:1702
title:231, distance:0.1484907865524292, predicted index:3858, actual index:3858
title:81671, distance:0.3992381691932678, predicted index:3335, actual index:735
title:81672, distance:0.37823259830474854, predicted index:769, actual index:769
title:231, distance:0.20698243379592896, predicted index:5624, actual index:5626
title:166, distance:0.4433469772338867, predicted index:3733, actual index:3726
title:5454, distance:0.15534847974777222, predicted index:1610, actual index:1610
title:81675, distance:0.2922547459602356, predicted index:754, actual index:753
title:81676, distance:0.36365997791290283, predicted index:172, actual index:172
title:4175, distance:0.160862982